# Strategy Details

Low volatility anomaly can be combined with the momentum anomaly to enhance risk-adjusted returns.

- The paper shows that incorporating both momentum and low volatility anomalies yields positive exposure to factors like value and profitability.

- Returns from these strategies are consistent over time and are more pronounced in later subsamples, with higher robust Sharpe Ratios.

- Plain momentum portfolios exhibit the highest robust Sharpe Ratio.

- For long-only investors, the DS strategy, which sorts stocks by momentum first and then by low volatility, seems superior to other strategies.

Note that the research has been conducted in the Nordic stock markets.

This paper investigates the profitability of combined low-volatility and momentum investment strategies in the Nordic stock markets from January 1999 to September 2022. Confirming earlier studies, our results first indicate that both the volatility and momentum effects persist as pure-play strategies. Further, we explore combined strategies using 50/50, double screening, and ranking strategies. Among the long-only portfolios, the momentum-first strategy generates the best Sharpe ratio using the double screening method−slightly outperforming the ranking method. Additionally, all long-only combination portfolios outperform the market in terms of risk-adjusted returns. Combination long-short strategies produce significantly higher risk-adjusted returns than pure-play strategies. Surprisingly, novel evidence suggests that none of the combination long- short strategies outperforms the pure momentum strategy after risk-adjusting the returns using the Fama and French five-factor model, implying that while momentum may enhance the returns from the low-volatility strategy, the reverse is not true for the Nordic stock markets.

As I encounter some problem with finding nordic stock markets, I use S&P500 to do backtesting first

In [1]:
# Collect the list of the S&P 500 companies from Wikipedia and save it to a file
import os
import requests
import pandas as pd
import numpy as np

# Get the list of S&P 500 companies from Wikipedia
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
response = requests.get(url)
html = response.content
df = pd.read_html(html, header=0)[0]

tickers = df['Symbol'].tolist()

In [2]:
# Load the data from yahoo finance
import os
import yfinance as yf

def load_data(symbol):

    direc = 'data/'
    os.makedirs(direc, exist_ok=True)

    file_name = os.path.join(direc, symbol + '.csv')

    if not os.path.exists(file_name):

        ticker = yf.Ticker(symbol)
        df = ticker.history(start='2005-01-01', end='2023-12-31')

        df.to_csv(file_name)

    df = pd.read_csv(file_name, index_col=0)
    df.index = pd.to_datetime(df.index, utc=True).tz_convert('US/Eastern')
    df['date'] = df.index

    if len(df) == 0:
        os.remove(file_name)
        return None

    return df

holder = []
ticker_with_data = []
for symbol in tickers:
    df = load_data(symbol)
    if df is not None:
        holder.append(df)
        ticker_with_data.append(symbol)

tickers = ticker_with_data[:]

print (f'Loaded data for {len(tickers)} companies')

BRK.B: No timezone found, symbol may be delisted
BF.B: No price data found, symbol may be delisted (1d 2005-01-01 -> 2023-12-31)
GEV: Data doesn't exist for startDate = 1104555600, endDate = 1703998800
SOLV: Data doesn't exist for startDate = 1104555600, endDate = 1703998800


Loaded data for 499 companies


In [38]:
print(len(tickers),len(holder))

499 499


In [3]:
# Get the monthly data
monthly_data = []
for data in holder:
    df = data.resample('M').agg({
        'date': 'first',
        'Open': 'first',
        'High': 'max',
        'Low': 'min',
        'Close': 'last',
        'Volume': 'sum'
    })

    df.set_index('date', inplace=True)

    monthly_data.append(df)


In [8]:
# Get the weekly data
weekly_data = []

for data in holder:
    df = data.resample('W').agg({
        'date': 'first',
        'Open': 'first',
        'High': 'max',
        'Low': 'min',
        'Close': 'last',
        'Volume': 'sum'
    })

    df.set_index('date', inplace=True)

    weekly_data.append(df)


## Add the monthly & weekly returns and the volatilties

Assume we will open at the monthly open price and will close at the monthly close. Or we will open at the weekly open price and will close at the weekly close

In [9]:
# Calculate the monthly returns
temp = []
for df in monthly_data:
    df['month'] = df.index.month
    df['year'] = df.index.year
    df['intra_month_return'] = df['Close'] / df['Open'] - 1
    df['next_intra_month_return'] = df['intra_month_return'].shift(-1)
    df['monthly_return'] = df['Close'].pct_change()
    df['next_month_return'] = df['monthly_return'].shift(-1)

    # Last 12 months return except the current month
    df['rolling_12_months_return'] = df['monthly_return'].rolling(11).sum().shift()

    temp.append(df)

monthly_data = temp

In [10]:
# Calculate the weekly returns
temp = []

for df in weekly_data:
    df['intra_week_return'] = df['Close'] / df['Open'] - 1
    df['next_intra_week_return'] = df['intra_week_return'].shift(-1)
    df['weekly_return'] = df['Close'].pct_change()
    df['next_weekly_return'] = df['weekly_return'].shift(-1)
    #df['rolling_12_weeks_return'] = df['Close'].pct_change().rolling(11).sum().shift()
    temp.append(df)

weekly_data = temp


In [12]:
# Calculate the standard deviation of weekly returns for each stock using the 3 years of historical data, recalculated every month
temp = []
for i, df in enumerate(weekly_data):

    df_monthly = monthly_data[i]
    df['rolling_36_months_std'] = df['weekly_return'].rolling(3*52).std()

    print (df.tail())
    print (df_monthly.tail())
    raise ValueError
    


    iteration = 52*3-1
    monthly_data[i]['rolling_36months_std'] = np.nan
    for row in range(len(monthly_data[i])):
        if row > 35:
            
            for j in range(iteration, len(weekly_data[i])):
                if weekly_data[i].index[j] >= monthly_data[i].index[row]:
                    #print(weekly_data[i].index[j],monthly_data[i].index[row])
                    #print(row)
                    #print(weekly_data[i].index[j],monthly_data[i].index[row])
                    iteration = j - 1
                    break
            monthly_data[i].at[monthly_data[i].index[row],'rolling_36months_std'] = weekly_data[i].iloc[iteration-156:iteration]['weekly_return'].std() 
            #print(monthly_data[i].iloc[row]['rolling_36months_std'], iteration)

                                Open       High        Low      Close  \
date                                                                    
2023-11-27 00:00:00-05:00  78.156261  81.583742  78.107417  81.290657   
2023-12-04 00:00:00-05:00  81.689584  85.410147  81.600030  84.156387   
2023-12-11 00:00:00-05:00  84.425044  87.624567  81.176682  87.095383   
2023-12-18 00:00:00-05:00  87.111669  87.445463  84.368052  86.566200   
2023-12-26 00:00:00-05:00  86.541783  89.700596  86.517357  89.000443   

                             Volume  intra_week_return  \
date                                                     
2023-11-27 00:00:00-05:00  22731655           0.040104   
2023-12-04 00:00:00-05:00  20927608           0.030197   
2023-12-11 00:00:00-05:00  37051960           0.031630   
2023-12-18 00:00:00-05:00  16004871          -0.006262   
2023-12-26 00:00:00-05:00  12502865           0.028410   

                           next_intra_week_return  weekly_return  \
date         

ValueError: 

# 8 Combinations

## 50/50

#### Momentum Portfolios

In [307]:
rolling_12_months_return_holder = []
next_intra_month_return_holder = []

# Creating tables with symbols as columns and the date as rows
for symbol, df in zip(tickers, monthly_data):

    rolling_12_months_return_series = df['rolling_12_months_return'].copy().dropna()
    next_intra_month_return_series = df['next_intra_month_return'].copy().dropna()

    rolling_12_months_return_series.name = symbol
    next_intra_month_return_series.name = symbol

    rolling_12_months_return_holder.append(rolling_12_months_return_series)
    next_intra_month_return_holder.append(next_intra_month_return_series)

rolling_12_months_return_df = pd.concat(rolling_12_months_return_holder, axis=1, ignore_index=False)
next_intra_month_return_df = pd.concat(next_intra_month_return_holder, axis=1, ignore_index=False)

print (rolling_12_months_return_df.iloc[:3, :4])
print (next_intra_month_return_df.iloc[:3, :4])



                                MMM       AOS       ABT  ABBV
date                                                         
2006-01-03 00:00:00-05:00 -0.047072  0.314975 -0.099370   NaN
2006-02-01 00:00:00-05:00 -0.109366  0.577854 -0.019359   NaN
2006-03-01 00:00:00-05:00 -0.111008  0.553794 -0.009187   NaN
                                MMM       AOS       ABT  ABBV
date                                                         
2005-01-03 00:00:00-05:00  0.001122 -0.032401  0.021546   NaN
2005-01-21 00:00:00-05:00       NaN       NaN       NaN   NaN
2005-02-01 00:00:00-05:00  0.020726  0.096468  0.013699   NaN
                                MMM       AOS       ABT  ABBV
date                                                         
2005-02-01 00:00:00-05:00  0.001003 -0.030616  0.021546   NaN
2005-03-01 00:00:00-05:00  0.020848  0.098555  0.013699   NaN
2005-04-01 00:00:00-05:00 -0.107597 -0.007227  0.060572   NaN


In [311]:
# Get the winner and loser portfolios based on rolling_12_months_return_df using tercile sorting
winner_momentum_stocks = {}
loser_momentum_stocks = {}

winner_loser_df = pd.DataFrame(index=rolling_12_months_return_df.index, columns=rolling_12_months_return_df.columns)
winner_loser_df.iloc[:, :] = 0

N = len(tickers) // 3
winner_mask = rolling_12_months_return_df.rank(axis=1, method='dense', ascending=False) > N
loser_mask = rolling_12_months_return_df.rank(axis=1, method='dense', ascending=True) > N

winner_loser_df[winner_mask] = 1
winner_loser_df[loser_mask] = -1

In [315]:
# Build an equal weighted momentum portfolio based on the winner stocks
returns_df_for_simulation = pd.DataFrame(index=rolling_12_months_return_df.index, columns=rolling_12_months_return_df.columns)
returns_df_for_simulation.iloc[:, :] = np.nan

returns_df_for_simulation[winner_mask] = next_intra_month_return_df[winner_mask]
returns_df_for_simulation[loser_mask] = next_intra_month_return_df[loser_mask] * -1

equal_weight_portfolio = returns_df_for_simulation.mean(axis=1, skipna=True)

# TODO: Complete this part
    


                           high_momentum_portfolio_returns
date                                                      
2006-01-03 00:00:00-05:00                              NaN
2006-02-01 00:00:00-05:00                         0.010055
2006-03-01 00:00:00-05:00                         0.040111
2006-04-03 00:00:00-04:00                         0.005596
2006-05-01 00:00:00-04:00                        -0.060606


C:\Users\bingy\AppData\Local\Temp\ipykernel_14952\2022198152.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.01005501357142311' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  high_momentum_portfolio_returns.loc[date, 'high_momentum_portfolio_returns'] = returns


#### Low-Volatility Portfolios

In [326]:
rolling_36months_std_holder = []

# Creating tables with symbols as columns and the date as rows

for symbol, df in zip(tickers, monthly_data):
        
        rolling_36months_std_series = df['rolling_36months_std'].copy().dropna()
        rolling_36months_std_series.name = symbol
        rolling_36months_std_holder.append(rolling_36months_std_series)

rolling_36months_std_df = pd.concat(rolling_36months_std_holder, axis=1, ignore_index=False)
rolling_36months_std_df.fillna(np.nan, inplace=True)

print (rolling_36months_std_df.iloc[:3, :4])


                                MMM       AOS       ABT  ABBV
date                                                         
2008-02-01 00:00:00-05:00  0.024560  0.045382  0.026341   NaN
2008-03-03 00:00:00-05:00  0.025347  0.046401  0.027145   NaN
2008-04-01 00:00:00-04:00  0.025392  0.046864  0.028078   NaN


In [368]:
# Sort the stocks based on the std of 36 weeks returns and get the low volatility stocks and high volatility stocks
low_volatility_stocks = {}
high_volatility_stocks = {}

for date, row in rolling_36months_std_df.iterrows():
    if date not in next_intra_month_return_df.index:
        continue
    temp = row.copy()
    # Convert temp to a dataframe
    temp = pd.Series(row)
    temp = temp.to_frame()

    # set the index to numbers
    temp = temp.reset_index()
    temp.columns = ['symbol','36 weeks std']
    
    # Sort the series in descending order
    temp.sort_values(ascending=True, inplace=True,by='36 weeks std')

    # Drop the nan values
    temp.dropna(inplace=True)

    low_volatility_stocks[date] = temp.iloc[:int(len(temp) / 3)]
    high_volatility_stocks[date] = temp.iloc[-int(len(temp) / 3):]

# Build an equal weighted momentum portfolio based on the low volatility stocks

low_volatility_portfolio_returns = pd.DataFrame(index=rolling_36months_std_df.index, columns=['low_volatility_portfolio_returns'])
low_volatility_portfolio_returns['low_volatility_portfolio_returns'] = np.nan

for date, row in low_volatility_stocks.items():
    if date not in next_intra_month_return_df.index:
        continue
    returns = next_intra_month_return_df.loc[date, row['symbol']].mean()
    low_volatility_portfolio_returns.loc[date, 'low_volatility_portfolio_returns'] = returns

low_volatility_portfolio_returns.dropna(inplace=True)
low_volatility_portfolio_returns['low_volatility_portfolio_returns'] = low_volatility_portfolio_returns['low_volatility_portfolio_returns'].shift()

print (low_volatility_portfolio_returns.head(5))

# Build an equal weighted momentum portfolio based on the high volatility stocks


high_volatility_portfolio_returns = pd.DataFrame(index=rolling_36months_std_df.index, columns=['high_volatility_portfolio_returns'])
high_volatility_portfolio_returns['high_volatility_portfolio_returns'] = np.nan

for date, row in high_volatility_stocks.items():
    if date not in next_intra_month_return_df.index:
        continue
    returns = next_intra_month_return_df.loc[date, row['symbol']].mean()
    high_volatility_portfolio_returns.loc[date, 'high_volatility_portfolio_returns'] = returns

high_volatility_portfolio_returns.dropna(inplace=True)
high_volatility_portfolio_returns['high_volatility_portfolio_returns'] = high_volatility_portfolio_returns['high_volatility_portfolio_returns'].shift()

print (high_volatility_portfolio_returns.head(5))



                           low_volatility_portfolio_returns
date                                                       
2008-02-01 00:00:00-05:00                               NaN
2008-03-03 00:00:00-05:00                          0.004810
2008-04-01 00:00:00-04:00                          0.029943
2008-05-01 00:00:00-04:00                          0.016431
2008-06-02 00:00:00-04:00                         -0.066477
                           high_volatility_portfolio_returns
date                                                        
2008-02-01 00:00:00-05:00                                NaN
2008-03-03 00:00:00-05:00                          -0.012658
2008-04-01 00:00:00-04:00                           0.075007
2008-05-01 00:00:00-04:00                           0.049648
2008-06-02 00:00:00-04:00                          -0.101154


### Long-Only

### Long-Short

## Double Screening

### Winner Stocks from Low-Volatility Universe

In [363]:
winners_from_low_volatility = {}
for date, stock in low_volatility_stocks.items():
    # Get the monthly returns of the low volatility stocks
    winners_from_low_volatility[date] = low_volatility_stocks[date]
    winners_from_low_volatility[date]['2-12 cumulative returns'] = 0
    winners_from_low_volatility[date] = winners_from_low_volatility[date].reset_index()
    
    temp = rolling_12_months_return_df.loc[date, stock['symbol']].copy()
    temp = temp.to_frame()
    temp = temp.reset_index()
    temp.columns = ['symbol','2-12 cumulative returns']
    winners_from_low_volatility[date]['2-12 cumulative returns'] = temp['2-12 cumulative returns']
    
    # Sort the series in descending order

    winners_from_low_volatility[date].sort_values(ascending=False, inplace=True,by='2-12 cumulative returns')
    # Keep the half of the stocks with the highest 2-12 cumulative returns
    winners_from_low_volatility[date] = winners_from_low_volatility[date].iloc[:int(len(winners_from_low_volatility[date]) / 2)]
    winners_from_low_volatility[date].reset_index(drop=True, inplace=True)




In [366]:
# Build an equal weighted momentum portfolio based on the winners_from_low_volatility

winners_from_low_volatility_portfolio_returns = pd.DataFrame(rolling_36months_std_df.index, columns=['winners_from_low_volatility_portfolio_returns'])
winners_from_low_volatility_portfolio_returns['winners_from_low_volatility_portfolio_returns'] = 0

for date, row in winners_from_low_volatility.items():
    if date not in next_intra_month_return_df.index:
        continue
    returns = next_intra_month_return_df.loc[date, row['symbol']].mean()
    winners_from_low_volatility_portfolio_returns.loc[date, 'winners_from_low_volatility_portfolio_returns'] = returns

winners_from_low_volatility_portfolio_returns['winners_from_low_volatility_portfolio_returns'] = winners_from_low_volatility_portfolio_returns['winners_from_low_volatility_portfolio_returns'].shift()

print (winners_from_low_volatility_portfolio_returns.head(10))


                           winners_from_low_volatility_portfolio_returns
2008-02-01 00:00:00-05:00                                            NaN
2008-03-03 00:00:00-05:00                                       0.005503
2008-04-01 00:00:00-04:00                                       0.025398
2008-05-01 00:00:00-04:00                                       0.017630
2008-06-02 00:00:00-04:00                                      -0.049774
2008-07-01 00:00:00-04:00                                       0.018596
2008-08-01 00:00:00-04:00                                       0.010984
2008-09-02 00:00:00-04:00                                      -0.058655
2008-10-01 00:00:00-04:00                                      -0.146239
2008-11-03 00:00:00-05:00                                      -0.052642


### Loser Stocks from High-Volatility Universe

In [370]:
losers_from_high_volatility = {}

for date, stock in high_volatility_stocks.items():
    # Get the monthly returns of the high volatility stocks
    losers_from_high_volatility[date] = high_volatility_stocks[date]
    losers_from_high_volatility[date]['2-12 cumulative returns'] = 0
    losers_from_high_volatility[date] = losers_from_high_volatility[date].reset_index()
    
    temp = rolling_12_months_return_df.loc[date, stock['symbol']].copy()
    temp = temp.to_frame()
    temp = temp.reset_index()
    temp.columns = ['symbol','2-12 cumulative returns']
    losers_from_high_volatility[date]['2-12 cumulative returns'] = temp['2-12 cumulative returns']
    
    # Sort the series in descending order

    losers_from_high_volatility[date].sort_values(ascending=False, inplace=True,by='2-12 cumulative returns')
    # Keep the half of the stocks with the highest 2-12 cumulative returns
    losers_from_high_volatility[date] = losers_from_high_volatility[date].iloc[:int(len(losers_from_high_volatility[date]) / 2)]
    losers_from_high_volatility[date].reset_index(drop=True, inplace=True)


In [371]:
# Build an equal weighted momentum portfolio based on the losers_from_high_volatility

losers_from_high_volatility_portfolio_returns = pd.DataFrame(rolling_36months_std_df.index, columns=['losers_from_high_volatility_portfolio_returns'])
losers_from_high_volatility_portfolio_returns['losers_from_high_volatility_portfolio_returns'] = 0

for date, row in losers_from_high_volatility.items():
    if date not in next_intra_month_return_df.index:
        continue
    returns = -next_intra_month_return_df.loc[date, row['symbol']].mean()
    losers_from_high_volatility_portfolio_returns.loc[date, 'losers_from_high_volatility_portfolio_returns'] = returns

losers_from_high_volatility_portfolio_returns['losers_from_high_volatility_portfolio_returns'] = losers_from_high_volatility_portfolio_returns['losers_from_high_volatility_portfolio_returns'].shift()

print (losers_from_high_volatility_portfolio_returns.head(10))


                           losers_from_high_volatility_portfolio_returns
2008-02-01 00:00:00-05:00                                            NaN
2008-03-03 00:00:00-05:00                                       0.018891
2008-04-01 00:00:00-04:00                                      -0.081978
2008-05-01 00:00:00-04:00                                      -0.050750
2008-06-02 00:00:00-04:00                                       0.071153
2008-07-01 00:00:00-04:00                                      -0.010763
2008-08-01 00:00:00-04:00                                       0.015535
2008-09-02 00:00:00-04:00                                       0.159534
2008-10-01 00:00:00-04:00                                       0.204237
2008-11-03 00:00:00-05:00                                       0.116889


### Low-Volatility Stocks from Winners

### High-Volatility Stock from Losers

## Ranking